### Project-01

In [13]:
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
# data loaders
from torch.utils.data import DataLoader
from torchvision import transforms

In [25]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("./data/train/")
valid_dataset = ImageFolder("./data/valid/")

In [27]:
train_dataset[0]

(<PIL.Image.Image image mode=RGB size=70x70 at 0x7F9F2F3E0BA8>, 0)

In [51]:
print(len(train_dataset))
print(len(valid_dataset))

3000
914


In [52]:
for i in range(0, len(train_dataset), 500):
    print(train_dataset.samples[i])


('./data/train/bill_gates/006f6dcf-315f-4291-80d2-b61054210273.jpg', 0)
('./data/train/bill_gates/9988e7f8-39c9-49e1-99c2-4d68e568c7d6.jpg', 0)
('./data/train/elon_musk/7efe6db3-061f-4850-b03e-841228a05d46.jpg', 1)
('./data/train/jeff_bezos/5808df03-ab8d-43cf-85e6-f8f4480a0949.jpg', 2)
('./data/train/mark_zuckerberg/4330dbea-f482-4ca9-b8a3-6beb318bc662.jpg', 3)
('./data/train/steve_jobs/22d92e6e-4df5-4025-8d6f-9f4b77b3b8f4.jpg', 4)


In [ ]:
trarget_name = ['Bill Gates', 'Elon Musk', 'Jeff Bezos', 'Mark Zuckerberg', 'Steve Jobs']

In [64]:
import random

def plot_samples(dataset, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(16, 8))

    for i in range(num_samples):
        random_image, random_class = random.choice(dataset)
        random_label = dataset.classes[random_class]
        axes[i].imshow(random_image)
        axes[i].set_title("Сelebrity: " + random_label.replace("_", " ").upper())
        axes[i].axis("off")

    plt.show()

In [66]:
#plot_samples(train_dataset)
#plot_samples(valid dataset)

In [68]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset.transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])
valid_dataset.transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [71]:
import os

BATCH_SIZE = 16

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=os.cpu_count()
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count()
)

In [72]:
for images, targets in tqdm(valid_loader):
    pass



100%|██████████| 58/58 [00:05<00:00, 10.79it/s]


### Train model

In [73]:
from torchvision import models

net = models.resnet34()

In [74]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [75]:
net.fc

Linear(in_features=512, out_features=1000, bias=True)

In [76]:
net.fc = torch.nn.Linear(512, len(train_dataset.classes))

In [77]:
criterion = torch.nn.CrossEntropyLoss()

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)
criterion = criterion.to(device)

In [79]:
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, weight_decay=1e-4)

In [80]:
def train_model(net, optimizer, train_dataset, valid_dataset, num_epochs=10):
    loss_train = []
    top1_train = []
    loss_valid = []
    top1_valid = []

    for epoch in range(num_epochs):
        # переключаем torch в режим сохрания данных для рассчета градиентов
        torch.set_grad_enabled(True)
        # переключаем модель в режим обучения для хранения промежуточной информация
        net.train()

        losses = []
        equals = []

        for i, (images, target) in enumerate(tqdm(train_loader)):
            # переносим на GPU при возможности
            images = images.to(device)
            target = target.to(device)

            # считаем сырые предсказания и функцию потерь
            output = net(images)
            loss = criterion(output, target)

            # запоминаем функцию потерь и правильность предсказаний
            losses.append(loss.item())
            equals.extend([eq.item() for eq in output.argmax(1) == target])

            # делаем шаг обновления весов модели
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_train.append(np.mean(losses))
        top1_train.append(np.mean(equals))

        # для валидации градиенты не нужны
        torch.set_grad_enabled(False)
        # переключаем модель в режим валидации (быстрее, не храним ничего лишнего)
        net.eval()

        losses = []
        equals = []

        for i, (images, target) in enumerate(tqdm(valid_loader)):
            # переносим на GPU при возможности
            images = images.to(device)
            target = target.to(device)

            # считаем сырые предсказания и функцию потерь
            output = net(images)
            loss = criterion(output, target)

            # запоминаем функцию потерь и правильность предсказаний
            losses.append(loss.item())
            equals.extend([eq.item() for eq in output.argmax(1) == target])

            # шаг обновления весов здесь не нужен
            ...

        loss_valid.append(np.mean(losses))
        top1_valid.append(np.mean(equals))

    return loss_train, top1_train, loss_valid, top1_valid

In [ ]:
loss_train, top1_train, loss_valid, top1_valid = train_model(net, optimizer, train_dataset, valid_dataset, 1)